<a href="https://colab.research.google.com/github/osipov/edu/blob/master/tf0/Demo_Keras_GCS_Tensorboard.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a>

## **TODO:** Set the value of `BUCKET` below to the BUCKET from your learning materials

In [ ]:
BUCKET = None
import os
assert BUCKET and (type(BUCKET) is str), "Be sure to initialize BUCKET using the value from your learning materials"
os.environ['BUCKET'] = BUCKET

### Authenticate using your Google Cloud account so you can access the dataset on Google Cloud Storage

In [ ]:
from google.colab import auth
auth.authenticate_user()

### Start with your usual imports

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.data as tfd

from tensorflow.data import Dataset

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

## Define 2 Python lists: 1st for the feature column names; 2nd for the target column name

In [ ]:
FEATURES = ['origin_block_latitude','origin_block_longitude','destination_block_latitude','destination_block_longitude']
TARGET = ['fareamount']

## Use `make_csv_dataset` to load a CSV dataset from GCS

In [ ]:
BATCH_SIZE = 2 ** 18

def glob_to_ds(glob, num_epochs = 1, batch_size = 1):
  def decode(x):
    yX = tf.stack(list(x.values()), axis = 1)
    return yX[:, 1:], yX[:, 0]
  
  ds = (tf.data.experimental
        .make_csv_dataset(glob, 
                          num_epochs = num_epochs,
                          batch_size = batch_size)
        .map(decode))
  
  return ds


[train_ds, 
 valid_ds, 
 test_ds] = [glob_to_ds(glob, 
                        batch_size = BATCH_SIZE) for glob in \
                        [f"gs://{BUCKET}/train/part-*.csv",
                         f"gs://{BUCKET}/valid/part-*.csv",
                         f"gs://{BUCKET}/test/part-*.csv"]]

train_ds, valid_ds, test_ds

## Create a model using `keras.Dense`

In [ ]:
model = Sequential()
model.add(Dense(units=13, input_shape=[len(FEATURES)], activation='relu'))
model.add(Dense(units=13, activation='relu'))
model.add(Dense(units=13, activation='relu'))
model.add(Dense(units=1, activation='linear'))

In [ ]:
model.summary()

## Create an instance of the `Adam` optimizer for the model

In [ ]:
LEARNING_RATE = 0.003
optimizer = Adam( learning_rate = LEARNING_RATE )

## Train using Keras `compile` and `fit` over the batches returned by your `BatchDataset` instance
* you can also use `tf.keras.metrics` to report on performance

In [ ]:
def train(epochs):
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
  mc_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath="model-epoch-{epoch:02d}-val-mse-{val_loss:.4f}",
      save_best_only=True, 
      monitor="val_loss",
      verbose=1,
  )

  model.compile(optimizer = optimizer, 
                loss = tf.keras.losses.MSE,  
                metrics = [tf.keras.metrics.RootMeanSquaredError(), 
                        tf.keras.metrics.MeanAbsoluteError()]) 


  training = model.fit(train_ds, 
                      validation_data = valid_ds,                     
                      epochs = epochs, 
                      callbacks=[tb_callback, mc_callback],
                      verbose = 1,)

  {key: training.history[key] for key in training.history.keys()}

train(epochs = 2)

## Load TensorBoard and render the training/validation logs

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
train(epochs = 100)

## Load the best performing model as a new instance of a Keras model

In [ ]:
models = !ls -1 -d -t $(find model* -maxdepth 0 -type d) | head -n 1
model = models[0] if len(models) > 0 else None
print(model)
best_model = tf.keras.models.load_model(model)
best_model

## Test the model performance

In [ ]:
results = best_model.evaluate(test_ds)

predictions = best_model.predict(test_ds)
predictions

Copyright 2021 CounterFactual.AI LLC. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.